In [1]:
import sklearn as sk
import pandas as pd
from transformers import pipeline

In [2]:
reviews = pd.read_csv('Desktop/dataset.csv')

In [3]:
reviews.describe()

,app_id,review_score,review_votes
count,6.417106e+06,6.417106e+06,6.417106e+06
mean,2.274695e+05,6.394992e-01,1.472446e-01
std,1.260451e+05,7.687918e-01,3.543496e-01
min,1.000000e+01,-1.000000e+00,0.000000e+00
25%,2.018100e+05,1.000000e+00,0.000000e+00
50%,2.391600e+05,1.000000e+00,0.000000e+00
75%,3.056200e+05,1.000000e+00,0.000000e+00
max,5.653400e+05,1.000000e+00,1.000000e+00


In [4]:
len(reviews)

6417106

In [5]:
showing = reviews.tail(100)
showing

,app_id,app_name,review_text,review_score,review_votes
6417006,99910,Puzzle Pirates,I wish I could give a 'meh.' Played this on a ...,-1,0
6417007,99910,Puzzle Pirates,This game was my childhood game... but when ...,-1,1
6417008,99910,Puzzle Pirates,"Pretty dull, and the animations/sound effects ...",-1,0
6417009,99910,Puzzle Pirates,"Too addictive, spent way too much time on this...",-1,1
6417010,99910,Puzzle Pirates,I lost years to this game but it's still the s...,-1,0
...,...,...,...,...,...
6417101,99910,Puzzle Pirates,I really ove this game but it needs somethings...,-1,0
6417102,99910,Puzzle Pirates,"Used to play Puzzel Pirates 'way back when', b...",-1,0
6417103,99910,Puzzle Pirates,"This game was aright, though a bit annoying. W...",-1,0
6417104,99910,Puzzle Pirates,"I had a nice review to recommend this game, bu...",-1,0


In [6]:
reviews['app_name'].unique().shape

(9364,)

Start with finding recurring words before we pull in ai

In [7]:

reviewss = reviews.head(10000)

In [8]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest

# SPACY MODEL
we used spacy Documentation as well as a script written here https://www.kaggle.com/code/itsmohammadshahid/nlp-text-summarizer-using-spacy


In [9]:
def textSummarizer(text, percentage):
    
    # load the model into spaCy
    nlp = spacy.load('en_core_web_sm')
    
    # pass the text into the nlp function
    doc= nlp(text)
    
    ## The score of each word is kept in a frequency table
    tokens=[token.text for token in doc]
    freq_of_word=dict()
    
    # Text cleaning and vectorization 
    for word in doc:
        if word.text.lower() not in list(STOP_WORDS):
            if word.text.lower() not in punctuation:
                if word.text not in freq_of_word.keys():
                    freq_of_word[word.text] = 1
                else:
                    freq_of_word[word.text] += 1
                    
    # Maximum frequency of word
    max_freq=max(freq_of_word.values())
    
    # Normalization of word frequency
    for word in freq_of_word.keys():
        freq_of_word[word]=freq_of_word[word]/max_freq
        
    # In this part, each sentence is weighed based on how often it contains the token.
    sent_tokens= [sent for sent in doc.sents]
    sent_scores = dict()
    for sent in sent_tokens:
        for word in sent:
            if word.text.lower() in freq_of_word.keys():
                if sent not in sent_scores.keys():                            
                    sent_scores[sent]=freq_of_word[word.text.lower()]
                else:
                    sent_scores[sent]+=freq_of_word[word.text.lower()]
    
    
    len_tokens=int(len(sent_tokens)*percentage)
    
    # Summary for the sentences with maximum score. Here, each sentence in the list is of spacy.span type
    summary = nlargest(n = len_tokens, iterable = sent_scores,key=sent_scores.get)
    
    # Prepare for final summary
    final_summary=[word.text for word in summary]
    
    #convert to a string
    summary=" ".join(final_summary)
    
    # Return final summary
    return summary

In [10]:
string_text = reviewss['review_text'].tolist()
string_list = [str(item) for item in string_text]

In [11]:
string_final = ', '.join(string_list)

In [24]:
final_summary = textSummarizer(string_final, 0.001)

In [25]:
print("#"*50)
print("Summary of the text")
print("Length of summarized text:",len(final_summary))
print("#"*50)
print()
print(final_summary)

##################################################
Summary of the text
Length of summarized text: 8757
##################################################

, still the best game, its nice, 10/10 would ♥♥♥♥ again, Fun and original, still played by proffesionals today, ─────────────────────────▄▀▄  ─────────────────────────█─█  ─────────────────────────█─█  ─────────────────────────█─█  ─────────────────────────█─█  ─────────────────────────█─█  ─────────────────────────█─▀█▀█▄  ─────────────────────────█──█──█  ─────────────────────────█▄▄█──▀█  ────────────────────────▄█──▄█▄─▀█  ────────────────────────█─▄█─█─█─█  ────────────────────────█──█─█─█─█  ────────────────────────█──█─█─█─█  ────▄█▄──▄█▄────────────█──▀▀█─█─█  ──▄█████████────────────▀█───█─█▄▀  ─▄███████████────────────██──▀▀─█  ▄█████████████────────────█─────█  ██████████───▀▀█▄─────────▀█────█  ████████───▀▀▀──█──────────█────█  ██████───────██─▀█─────────█────█  ████──▄──────────▀█────────█────█ My gameplay was on anothe

In [26]:
import re

def remove_non_words(text):
    # Use regex to keep only word characters (letters, digits, and underscores)
    cleaned_text = re.sub(r'(?:&lt;3|[\W|\ʖ|])+', ' ', text)   # \W matches any non-word character
    return cleaned_text.strip()  # Remove leading/trailing whitespace

result = remove_non_words(final_summary)
print(result) 


still the best game its nice 10 10 would again Fun and original still played by proffesionals today My gameplay was on another edition not from steam so don t pay attention the gameplay hours 10 10 222 10 10 Easier to hack on One of the best game ever xdd Cs for life o old but good better then csgo One of the best game ever xdd Nice game Best shooter of this century Oooooh it s classic French Spanish Awesome game of all the times Game from my childhood Classic One of if not the best game of all time for me Been playing since the release awesome game so much fun can t play with bot ValveThank U The best game in the world Most of my friends are addicted to this game and it s like a drugs when you start to play you can t stop Relly nice game this game made my childhood its nice one An old classic but is still fun to play hmmmm it s a game Really Addictive Game With Great Players over 10 years still the best fps worth playing THIS GAME RUINED MY LIFE First online game ever that i loved thi

## OTHER EXAMPLES

In [15]:
text = """Stellaris Review
Some great ideas get lost in interstellar bureaucracy.

Rowan Kaiser Avatar
BY ROWAN KAISER
UPDATED: SEP 29, 2021 3:30 PM
POSTED: MAY 9, 2016 8:00 AM
The promising new space strategy game Stellaris has the pedigree to be great. With the experts at Paradox putting their own spin on a classic genre, it seems like a can’t-miss proposition. But it does miss, turning great early-game potential into a slow, dull grind.

Stellaris makes a great first impression. The empires in each game are randomly generated to have their own species traits, backgrounds, and government types. By far the most interesting twist is a set of ideology scales, with four ranges of Xenophobe-Xenophile, Spiritual-Materialist, Collectivist-Individualist, and Militarist-Pacifist determining how they behave. So you could create the Vulcans from Star Trek with Fanatic Materialism and Pacifism, or an angry swarming hive of Fanatic Collectivist Xenophobes to mimic Master of Orion’s Klackons. These decisions are meaningful enough to offer slight buffs or debuffs to most aspects of a campaign, from population happiness to diplomatic buffs or penalties with other races. With dozens of empires in any given galaxy (24 in a normal-sized startup) the randomization is flexible and strong enough to make the early empire-building fascinating.


The early game is divided into two main parts: first, you send you science ships to explore new systems, finding quests and new planets to inhabit. This works well: exploring is fun, and the quests you uncover are well-written and react well to empire’s ideologies. My favorite moment in all my 80-plus hours thus far with Stellaris involved discovering another race’s lost sacred text, but my Xenophobic race thought it was poorly written and refused to give it back out of spite. That caused a permanent negative penalty to my diplomacy with that race for the rest of the game. Arbitrary and random? Sure - but this rift between empires has a memorable story behind it.
Moving the pops around a planet to maximize resources is simple and surprisingly satisfying.
“
The second component of the early game involves deciding how to grow your empire. It’s a more detailed take a fairly conventional strategy game model, in that you must manage the populations (or ”pops”) of your planets by putting them to work on food or minerals or energy, then deciding whether to build factories, clear land for future construction, or send out colony ships. Moving the pops around a planet to maximize resources is simple and surprisingly satisfying, and doing this across a few planets keeps the early game moving quickly. What gives it depth and complexity is that each pop also has an ideology which can slowly change; so, for example, if you start developing Individualists, they’ll be unhappy to share land next to slaves.

There are some neat ideas involved in colony mechanics as well: your starting race is only adapted to life on a few types of worlds, so a species of ocean-dwellers simply won’t be happy on a desert planet. In most games that would limit your growth, but in Stellaris you can find primitive races on other worlds and either conquer and enslave them, or uplift them into your empire as citizens – or you can send in the robots. Science fiction games rarely portray multi-species empires, but Stellaris makes them essential.
So there’s a strong beginning, but then it all goes wrong.
“
So there’s a strong beginning, but once you get five planets or so into a decent-sized empire….then it all goes wrong. In the transition from the early game to the mid-game, Stellaris grinds to a halt. The vast majority of my time playing was spent staring at the screen, waiting for something, anything interesting to happen. It usually didn’t. And campaigns can last for dozens of hours.

Part of this is the fault of the overly passive AI. If they’re stronger than you, they might declare war. If they’re not, they don’t do anything. It took me 50 hours and three games, progressively raising the difficulty and giving a computer empire a huge buff before they declared war on me for the first time. Multiplayer is obviously preferable, but solo players will suffer – and that’s the primary way most people are likely to experience a game that requires such a massive time commitment.

Adding to this problem is how Stellaris’ diplomatic system is simplistic and frustrating in equal measure. It’s built for grand alliances with exciting names like the Harmonious Axis or Galactic Concord – and if the alliances are strong, they can be turned into Star Trek-like Federations with special ships taking the best of each member race’s tech.
The diplomatic conception of war in Stellaris doesn’t work well at all.
“
The problem is that alliances are built for war, and the diplomatic conception of war in Stellaris doesn’t work well at all. Essentially, the issue is that the goals you set before a war are too big: things like capturing or freeing an enemy colony or vassalizing an enemy empire. There are no options for something smaller-scale, like shifting a border star system into your sphere of influence in order to get that tundra planet for colonization. It all feels all-or-nothing. But resources are balanced by planet, so you don’t really need massive expansion except for its own sake. I found myself declaring war out of boredom rather than for any strategic benefit – never a good sign in any grand strategy game.

An especially annoying quirk of the alliance system is that it’s impossible to get out of wars your alliance members dragged you into. (Maybe it’s a little too real there.) In one game, I had a powerful enemy empire declare war on the weakest member of my alliance, even though all their wargoals were my planets - they attacked my little brother to get to me. They won the war, and slowly worked their way across my empire, destroying my entire infrastructure, and I couldn’t surrender to get out of it because I wasn’t the empire who was initially targeted.. I spent an hour or two just watching in increasing frustration before giving up and starting a new game.
Trade and espionage don’t exist in Stellaris.
“
There also aren’t any significant non-war reasons for engagement with foreign empires. Trade and espionage don’t exist in Stellaris. Trade in particular seems like a missed opportunity given Stellaris’ systems, as trade could set up chaotic intersections of species and ideologies. There is resource trading between empires, but it’s so clunky to use and has so little information available that it’s not worth it except in desperate situations.

Endgame catastrophes like robot rebellions or massive invasions are supposed to apply pressure and give reasons to take action other than conquest. But they don’t appear in every game, or appear early enough to prevent in-game boredom. Maybe it was just bad luck on my end, but I played multiple campaigns for at least 20 hours each, and never faced any surprises.
Sectors are black holes where fun gets sucked into an event horizon.
“
Not only is Stellaris externally inert, its internal empire management turns dull as well. Once you expand past five star systems, you’re penalized with a mechanic designed to prevent micromanagement. You’re forced to create sectors: clumps of your empire given over to an AI governor in order to give you resources without hassle. Unfortunately, these sectors are black holes where fun gets sucked into an event horizon, never to return.

Sectors take the best part of Stellaris – planet management – and turn into...nothing. They don’t do anything except contribute resources. Most importantly, they don’t create compelling internal politics to an empire – at worst, a discontented sector might threaten to rebel, but that’s easily stopped with a tiny payment. Ideologies would seem to be a great way to keep internal politics fresh, but they have such a minor effect at the macro level that they’re easy to ignore. Developer Paradox struck gold in its previous game, Crusader Kings 2, by turning intra-empire politics into essential gameplay, so it’s astonishing how utterly boring a seemingly similar mechanic is in Stellaris.

That’s a frustrating process that actually creates more micromanagement.
“
It gets worse! Putting planets in sectors hides them in the menus that show lists of planets, so there’s no way to quickly access them. And planets in sectors don’t build ships or improve their space stations on their own, so if you ever want to do that you have to find them on the map, and that’s a frustrating process that actually creates more micromanagement than the sectors solve.

Even the quest chains, which are so promising in the early game, simply stop appearing in the mid-game – and the long-term ones get stuck behind AI borders or broken by, say, a rival empire destroying a fleet you were supposed to destroy without Stellaris registering what happened. The only thing I found to look forward to at all was getting new technology, and even that has its issues.

Stellaris uses a neat system of drawing “cards” for each new tech, so you have a few options to choose from every time you advance in its three categories: Engineering, Physics, and Society. So, for example, a Society draw might see you have an option to terraform mountains into useable land, or buff your empire’s borders, or be able to colonize a new kind of planet. Whichever ones you don’t take, you risk never getting at all if they don’t appear in another drawing. At their best, this creates some tough choices.

The flip side of this is that almost all the interesting choices are in the Society category. Physics is occasionally useful, but Engineering is almost always a slight improvement to your ships. That sounds like it might be useful, but it’s incredibly hard to get excited about a new gun since combat, as in most Paradox games, is fully automated. Thus, whether you have a railgun or a mass driver makes no perceptible difference beyond one making the ship’s power rating go up.

At the strategic level, meanwhile, wars are solid but rarely exciting. Far too much time is spent chasing enemy ships down and forcing them into combat (or playing cat-and-mouse with your own fleets).
Verdict
Stellaris is filled with good ideas, and it’s not difficult to see the outline of a great space strategy game where those ideas could come together. But beyond the early game, it’s only compelling in bits and pieces – it turns into a largely uneventful slog after that. Paradox has developed a reputation of major upgrades to their games for years after launch, and Stellaris is going to need all that love and more to reach its potential.

God of War Ragnarok Review
God Týr

Simon Cardy Avatar
BY SIMON CARDY
UPDATED: NOV 18, 2022 9:39 PM
POSTED: NOV 3, 2022 11:00 AM
How do you follow up on one of the greatest video games of all time? Sony Santa Monica finds itself in a situation not dissimilar to when Francis Ford Coppola created the sequel to his mob movie masterpiece, The Godfather. Like part two of the Corleone story, God of War Ragnarok puts a fierce, younger member of the family directly under the microscope. In doing so it manages to reach the heights of its predecessor and, in some ways, even tower above it. The writing, performances, and music are each exceptional, bringing this expansive Norse tapestry to life – but even as it holds your heart in one hand with its elegantly told story it’s crunching bones in the other with fantastically ferocious combat. It all binds together to forge a monumental action epic that adds yet another impressive landmark to the video game landscape.

Keeping things broad and spoiler-free, Kratos' story picks up a few years after the final revelations of 2018's God of War – this is absolutely not one of those sequels where everything will make sense without playing the first game, or at the very least watching the included story recap (which is probably too brief to serve as anything but a jolt to the memory). The apocalyptic Ragnarok is coming, and its events circle around his son Atreus like one of Odin's ravens. Fimbulwinter – a time of unrest that presages Ragnarok – has very much come and, during the intervening years, Atreus has been growing stronger, learning about his Giant name "Loki", and trying to convince his father to trust him. There are touching callbacks to the hunting scene from the 2018 God of War as Kratos proudly watches Atreus prove that he’s no longer a child. But it’s no slow introduction: Ragnarok wastes no time in stepping into the action and, after a breakneck opening battle, the first step of this new journey is to leave the frozen realm of Midgard and find a missing Norse god in the broader and varied universe.


God of War Ragnarok Preview


18 IMAGES




IGN's Twenty Questions - Guess the game!
💡 Hint
To start:
...try asking a question that can be answered with a "Yes" or "No".
Ask a question to start guessing
000/250
Ask
It may not be as tightly told a story as 2018's, but Ragnarok has much grander designs. This is a sprawling epic that always keeps things on the move without ever becoming disorientating, gluing me to the edge of my seat as it repeatedly toyed with expectations. It's an ambition regularly met by stunning execution, with a momentum that never slips as you hurtle towards its bombastic ending. This journey took me just over 28 hours with a few fair side quests completed along the way, but with plenty still left to do afterward.

Prophecy and sacrifice swirl at the story’s centre as it repeatedly prods at the concept of destiny and whether blood runs thicker than the ink that pens it. It doesn’t merely offer up surface-level readings of these themes; instead, each character and their motives are given the respect they deserve by delving in deep. It’s a truly special and fittingly grand conclusion of Kratos’ Norse saga, as well as providing resolution to some of his more troubled Greek memories.

It’s a truly special and fittingly grand conclusion of Kratos’ Norse saga.
“
There are twists and turns aplenty, with fakeouts and misdirection woven into the story’s fabric as it darts between loud spectacle and tender conversation. Characters old and new play their own crucial parts, including fresh characterisations of Norse behemoths like Thor and Odin, with a line of dialogue rarely wasted between them. These are truly unique depictions, and it's a credit to Sony Santa Monica and the actors playing each that I wasn’t reminded once of their MCU equivalents. Despite being laden with myth and melodrama, there's still plenty of time for welcome pockets of self-aware silliness that almost always elicit a laugh, whether that be another bizarre dwarven contraption or the simple thrill of Kratos eating a sausage.

When not eating dinner, Kratos’ thunderous voice actor Christopher Judge frequently engages in smartly realised dialogue with his equally impressive castmates. Make no mistake, though: this is firmly Judge's court, and one he frequently dominates. Kratos' traumatic personal history is reflected in everything from his mournful tone of voice to his appearance - his Icarus wings are long-since clipped, but his complexion is still pale like a square of wall where a family portrait used to hang. Death weighs heavy in the winds that guide him on every journey he takes, whether delivering it himself or searching for some closure. The everlasting cycle of violence between parents and their children has long been a theme ever since the original God of War trilogy, although one that was only truly emphasised in 2018’s revival. Now it’s an inescapable fixture of Ragnarok; an exploration of the rawest and most human of emotions, and what happens when they are channeled through superhuman vessels.



Pain leads to anger, anger leads to hate – this is the genesis of the stalking threat embodied by the incomparable force that is a grieving mother. Portrayed incredibly with a newfound fury by Danielle Bisutti, Freya is a striking presence to be reckoned with in the sequel and casts a much darker shadow this time around. No longer just a helpful witch in the woods, she’s given a whole new spectrum of emotional depth to play in, and Ragnarok is all the better for it.

Each character is given their own moments to shine throughout; Sunny Suljic fires back regularly with the sharp, teenage tongue of a brasher, more confident Atreus, and Alastair Duncan’s severed head accessory Mimir continues to provide levity and exposition in abundance. Various newcomers to the vastly expanded cast also deliver fantastic performances, with a particular standout being Laya DeLeon Hayes, radiating infectious energy from the mysterious Angrboda. Ben Prendergast brings warmth to the nuanced Týr – he's charmingly plucky as the Norse God of War. The foul-mouthed Brok and skittish Sindri continue to bring their unique brand of dwarven charm while granting you those all-important upgrades. They often provide pockets of humour, as well as a lot of genuinely helpful info along the way as they play a much more central role in the story this time around.

Also returning is composer Bear McCreary, whose score – like the dwarven smith brothers – goes hammer and tongs from the first moment to the last. There's some truly beautiful music to be heard along the journey, as well as explosions of raw orchestral power during intense fight scenes. The artistry bursting from every seam is astonishing – even when running in the high frame-rate favouring Performance mode – with excellent costume and world design on show. Ragnarok is incredibly well polished, with little details from the etchings on a dagger to the clunk of a cupboard door closing shown love. It’s expertly animated, with saliva-sputtering grimaces launching towards the lens during the heat of battle, and hair flowing realistically on Kratos’ less-follically challenged enemies.


Every 10/10 Game Review IGN Has Ever Given

<b>The Legend of Zelda: Ocarina of Time (N64)</b><br>
Reviewed by Kristine Steimer<br />
November 25, 1998<br />

"The new benchmark for interactive entertainment has arrived. The Legend of Zelda: Ocarina of Time, the fifth official installment in Nintendo's popular action adventure series is finally here, and like its NES predecessor in 1987 it is a game so enjoyable, it has the power to pull videogame players into its imaginative worlds -- and refuse to let go for days. Call us crazy, but when the final version of Zelda 64 arrived in the IGN64 offices, we stopped working, locked ourselves into a room with a big-screen TV and a surround system and played 17 hours straight. After only a few hours of sleep, we were back for more and we couldn't stop until we finished the game. Then, we started over again to find all the secrets. Rarely is there such a perfect mixture of graphics, sound, and gameplay that even the most cynical players will admit that Zelda 64 is poised to shape the action RPG genre for years to come."
66 IMAGES
<b>Pokemon Red</b><br>
Reviewed by Craig Harris<br />
June 23, 1999<br />

"Pokemon really is worthy of being one of the Game Boy system's greatest games - it's not just a fad, a craze, or trend. If you don't have this game in your library, pick one up, it doesn't matter if you pick Blue or Red, they're both the same game. You'll just have to make sure you've got a friend with the other version if you want to catch 'em all."
<b>Pokemon Blue</b><br>
Reviewed by Craig Harris<br />
June 23, 1999<br />

"Pokemon really is worthy of being one of the Game Boy system's greatest games - it's not just a fad, a craze, or trend. If you don't have this game in your library, pick one up, it doesn't matter if you pick Blue or Red, they're both the same game. You'll just have to make sure you've got a friend with the other version if you want to catch 'em all."
<b>Checkered Flag</b><br>
Reviewed by Robert A. Jung<br />
July 6, 1999<br />

"This is a high-performance title that does nothing but pure video racing, and does it extremely well. Crammed with exciting gameplay, fast action, options galore, hot sound and graphics, and true multiplayer challenge, Checkered Flag is absolutely terrific!"
<b>Joust</b><br>
Reviewed by Robert A. Jung<br />
July 6, 1999<br />

"Joust is another unbelievably excellent product from Shadowsoft. Players who never played the arcade title will be captivated by its simple rules, deep strategy, and fast action, while the legions of arcade Joust fans will be in video-gaming heaven!"
Layered writing, impeccable acting, and stunning set design are all crucial aspects in driving God of War’s filmic aspirations home, but they’re also never let down by the rock-solid technical craftsmanship at work. The one continuous camera shot style of 2018's reboot returns, snaking through the entirety of Ragnarok as if mimicking the world serpent Jörmungandr himself. The technique may have lost a little of its initial wow factor four years on, but it's still a remarkably impressive achievement that has several new tricks to show off, and only adds to Ragnarok's cinematic nature.

A Song of Ice and Fire
God of War’s combat has always been the flashy attraction that draws a crowd in to be wowed by its story, however, and Ragnarok continues that tradition. It may seem familiar at first – and indeed some things remain the same – but it isn't afraid to play with expectations and throw a curveball your way. Kratos’ blades whip around in every direction, delivering a clanking firey pain to any face they meet. Heavy cleaves of his axe crunch into enemies, making them fold like bloodied origami. And yes, that axe still slices through the air, meatily lodging itself into anything unfortunate enough to be in its path, before being recalled to your palm with a hearty kick from the DualSense controller’s haptics in what remains one of the most satisfying button presses ever assigned to a controller. It's all unapologetically fast, and undeniably glorious – harking back to the brand of frantic action in which the series was originally rooted.

It's all unapologetically fast, and undeniably glorious – harking back to the brand of frantic action in which the series was originally rooted.
“
There’s a fair helping of excessive violence on show, with Kratos willfully impaling and ripping apart any enemy put in front of him. Arenas are often tight spaces full of threats, and though you aren’t going to see the return of dozens of enemies on screen at once that spawn in wave after wave, that doesn’t mean there aren’t some aspects of those older games shining through. But even with that slight shift to old-school action, Ragnarok’s eyes are still drawn more to the shimmer of Nathan Drake's gold rather than Bayonetta's platinum. Cinematic spectacle takes precedence over the chaotic, bordering-on cartoonish action of old.

The axe feels largely similar to how you remember (and that’s no bad thing) with its dynamic mix of heavy attacks and ranged power, but how you wield the blades is noticeably different. There's a greater emphasis on using them to hook enemies and maneuver them around the arena – whether that be dragging them in towards you or juggling them in the air like the hottest of potatoes. They're much closer to how they felt in the original games; a nostalgic move set with a brilliant modern edge. The action feels and looks outstanding and there are no caveats or exceptions when I say that Ragnarok is only an improvement over God of War’s already ferocious combat.


Both weapons have new tricks to show off, too: you can “Frost Awaken” the axe by holding in the triangle button to power up your next attack, or mash the same button while the blades are equipped to ignite them. Locking down an enemy by freezing them with the axe before switching to the blades to unleash pain on their friends is an endless source of enjoyment. Even the shield is now more of an offensive tool, and different types can be bought and crafted to fit your playstyle. A sturdier one can soak up heavier hits, while a high-risk/reward version boosts your parry's effectiveness. Parrying has a delicious weightiness to it, and when you nail the timing, it still feels fantastic after the thousandth deflect. Trust me when I say that's a skill you'll want to perfect before taking on some of Ragnarok's toughest fights, as well.

Runic abilities (special cooldown attacks assigned in your inventory) also return and, while there isn’t the same abundance of choice this time around, each is devastatingly effective. For example, Hades Retribution – a fiery stab that lodges a bomb into the ribcage of an enemy – is particularly fun to put into action. Brutal finishers on stunned enemies are another enjoyable throwback to the God of War of old. Heads are dismembered, waists are left standing and spouting blood, and torsos are carved in two. There's noticeably a good amount of variety in the randomly chosen, weapon-specific animations, too, so you won't get bored of seeing that one head stomp execution move from 2018 over and over again. My favourite has to be the simple pleasure of seeing the shock on a soldier’s face as their arms are hacked off one by one. Beautiful. Most importantly, all of these additions give an extra tactical edge as you mix up the cadence of your strikes and control each fight.

Kratos is a little lighter on his feet this time around, with a bit less of a thumping clomp to his sprint. Movement is further emphasised by the way you can now grapple up from a distance using the blades, and launch across gaps to dive into combat. Taller arena design literally adds extra layers to battles, allowing you to leap from a ledge to deliver death from above. Combat sequences can feel akin to Doom kill boxes as you dash and swing around elevated platforms toward your next victim in performances of acrobatic bloodshed – juggling cooldowns, Atreus’ arrows, and your rage meter to efficiently and spectacularly destroy everyone sent to stop you.
"""


In [16]:
import re

def remove_non_words(text):
    # Use regex to keep only word characters (letters, digits, and underscores)
    cleaned_text = re.sub(r'(?:&lt;3|[\W|\ʖ|])+', ' ', text)   # \W matches any non-word character
    return cleaned_text.strip()  # Remove leading/trailing whitespace
texts = remove_non_words(text)
textSummarizer(texts, 0.1)


'The new benchmark for interactive entertainment has arrived The Legend of Zelda Ocarina of Time the fifth official installment in Nintendo s popular action adventure series is finally here and like its NES predecessor in 1987 it is a game so enjoyable it has the power to pull videogame players into its imaginative worlds and refuse to let go for days Call us crazy but when the final version of Zelda 64 arrived in the IGN64 offices we stopped working locked ourselves into a room with a big screen TV and a surround system and played 17 hours straight After only a few hours of sleep we were back for more and we couldn t stop until we finished the game Then we started over again to find all the secrets Rarely is there such a perfect mixture of graphics sound and gameplay that even the most cynical players will admit that Zelda 64 is poised to shape the action RPG genre for years to come 66 IMAGES b Pokemon Red b br Reviewed by Craig Harris br June 23 1999 br Pokemon really is worthy of be

In [17]:
import re

def remove_non_words(text):
    # Use regex to keep only word characters (letters, digits, and underscores)
    cleaned_text = re.sub(r'(?:&lt;3|[\W|\ʖ|])+', ' ', text)   # \W matches any non-word character
    return cleaned_text.strip()  # Remove leading/trailing whitespace
reviewss = reviews[reviews['app_name'] == 'Puzzle Pirates']
reviews_text = reviewss['review_text'].tolist()
reviews_list = [str(item) for item in reviews_text]
reviews_str = '. '.join(reviews_list)
final_text = remove_non_words(reviews_str)
textSummarizer(final_text, 0.002)


'The game is still much more deeper than that the game is built on the shoulders of its players with a constant world heavily impacted by the politics of massive crews and flags and the entrepreneurship of individuals The path you choose is entirely up to you As many will find you will probably start off getting jobs aboard ships but this can quickly branch off into what YOU would like to do You can continue to keep jobbing for crews or join your own doing the same thing work at shoppes run your own pillages as an officer in a crew become a trader sailing island to island become an adventurer or monster hunter sailing to unknown dangers and looting giants join the polticial scene among many of the top flags and crews who run a good fraction of the ocean or even conquering entire islands owned by other players It s presented with a 2D setting and environment but its very stylized and clean The only true downside is the lack of effort put in by the developer s interest the game has const

AI was used to help with certain functions to fix errors as well as use the re library